<a href="https://www.kaggle.com/code/pearsejim01/fiverr-spam-2nd-in-publiclb-and-privatelb?scriptVersionId=113393402" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-potential-spammers-on-fiverr/sample_submission.csv
/kaggle/input/predict-potential-spammers-on-fiverr/train.csv
/kaggle/input/predict-potential-spammers-on-fiverr/test.csv


In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split#For splitting
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier, Pool 
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)


In [3]:
train = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/train.csv')
test = pd.read_csv('/kaggle/input/predict-potential-spammers-on-fiverr/test.csv')


In [4]:
def optimizedata(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
optimizedata(train)

Memory usage of dataframe is 185.52 MB
Memory usage after optimization is: 29.32 MB
Decreased by 84.2%


,label,user_id,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51
0,0,1,20972,14,13,3,11,1,2,15,26,48955,0,0,0.0,0,0,10,2,7,0,1,12,11,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,2,7362,213,71,3,11,1,2,15,26,48955,0,0,0.0,0,0,10,2,9,0,1,12,12,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,21216,215,71,3,11,1,2,15,14,48955,0,0,0.0,0,1,14,0,11,0,1,11,11,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,4,2261,212,71,2,8,1,2,15,14,48909,0,0,0.0,0,0,11,0,7,0,1,13,13,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,5,4543,213,71,2,8,1,2,15,14,48955,0,0,0.0,0,0,13,1,11,0,1,25,19,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458793,0,482931,1,213,13,3,16,4,4,22,27,47159,0,0,0.0,0,0,12,3,11,4,1,16,14,5,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
458794,0,482932,6139,211,13,3,11,4,4,22,28,43936,0,0,0.0,0,0,7,0,5,1,1,16,14,9,0,0,0,0,1,0,0,2,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
458795,0,482933,1164,213,13,3,11,4,4,22,26,1,0,0,0.0,0,1,10,0,10,0,0,39,27,3,1,1,0,0,1,0,0,1,1,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,0
458796,0,482934,187,137,14,2,2,4,4,22,14,7193,0,0,0.0,0,0,15,0,11,0,1,17,12,1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [6]:
optimizedata(test)

Memory usage of dataframe is 9.58 MB
Memory usage after optimization is: 1.45 MB
Decreased by 84.9%


,user_id,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51
0,57,19364,213,51,2,8,1,2,15,26,48955,0,0,0,0,0,10,3,10,0,1,24,20,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,58,16000,212,13,3,11,1,2,15,26,48955,0,0,0,0,0,10,1,9,0,1,22,15,1,0,1,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,59,14482,78,51,2,8,1,2,15,26,49002,0,0,0,0,0,7,0,6,0,1,17,14,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,60,18074,175,51,2,8,1,2,15,26,48955,0,0,0,0,0,10,1,6,0,1,28,19,2,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,61,4807,213,71,3,11,1,2,15,18,48955,0,0,0,0,0,11,1,9,0,1,19,14,3,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24143,482942,8559,152,58,3,16,4,4,5,28,23076,0,0,0,0,0,11,4,8,1,1,26,16,10,0,0,0,0,0,0,0,1,6,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
24144,482943,8124,213,28,3,16,4,4,22,36,49234,0,0,0,0,0,20,0,10,0,1,16,12,0,0,0,0,0,0,0,0,1,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24145,482944,12615,95,14,2,2,4,4,22,14,42275,0,0,0,0,0,12,3,9,0,1,16,13,6,0,0,0,0,1,0,0,1,4,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
24146,482945,8708,45,13,3,16,4,4,3,1,6038,0,0,0,1,0,12,0,12,0,1,112,23,15,0,6,0,0,3,0,0,1,1,0,1,1,0,0,0,0,0,5,0,0,5,0,0,0,0,0,0,0


In [17]:
train1=train.dropna()
test2 = test.drop(['X38', 'X34', 'X32', 'X24', 'X27', 'X29', 'X30', 'X33', 'X46'], axis=1)
train1['X13'] = train1['X13'].astype(int)


In [11]:
y1 = train1.label
X1 = train1.drop(['label', 'X38', 'X34', 'X32', 'X24', 'X27', 'X29', 'X30', 'X33', 'X46'], axis=1)
print(X1.columns, X1.shape)

Index(['user_id', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',
       'X21', 'X22', 'X23', 'X25', 'X26', 'X28', 'X31', 'X35', 'X36', 'X37',
       'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X47', 'X48', 'X49',
       'X50', 'X51'],
      dtype='object') (458792, 43)


In [34]:
categorical_features_indices =np.where((X1.dtypes != np.float32) | (X1.dtypes != np.float16))[0]
categorical_features_indices=np.delete(categorical_features_indices, 0)

categorical_features_indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42])

In [35]:
cat = CatBoostClassifier(scale_pos_weight = 2.01,eval_metric= 'AUC') 
train_pool = Pool(X1, y1, cat_features=categorical_features_indices)
cat.fit(train_pool)
cat_p = cat.predict(test2)
#highest is 2.01

Learning rate set to 0.141073
0:	total: 2.1s	remaining: 35m 1s
1:	total: 3.79s	remaining: 31m 32s
2:	total: 5.4s	remaining: 29m 55s
3:	total: 7.43s	remaining: 30m 49s
4:	total: 9.46s	remaining: 31m 23s
5:	total: 11.1s	remaining: 30m 45s
6:	total: 13s	remaining: 30m 39s
7:	total: 14.4s	remaining: 29m 51s
8:	total: 16.2s	remaining: 29m 43s
9:	total: 18.3s	remaining: 30m 15s
10:	total: 20.8s	remaining: 31m 7s
11:	total: 23.2s	remaining: 31m 50s
12:	total: 25.5s	remaining: 32m 15s
13:	total: 27.9s	remaining: 32m 41s
14:	total: 29.6s	remaining: 32m 21s
15:	total: 32.1s	remaining: 32m 57s
16:	total: 34.1s	remaining: 32m 53s
17:	total: 36s	remaining: 32m 42s
18:	total: 37.7s	remaining: 32m 26s
19:	total: 39.5s	remaining: 32m 16s
20:	total: 41.7s	remaining: 32m 23s
21:	total: 43.9s	remaining: 32m 33s
22:	total: 46.3s	remaining: 32m 47s
23:	total: 47.8s	remaining: 32m 24s
24:	total: 49.6s	remaining: 32m 14s
25:	total: 51.3s	remaining: 32m 3s
26:	total: 53.1s	remaining: 31m 53s
27:	total: 55.3s	

In [37]:
submission = pd.DataFrame(columns=['user_id', 'prediction'])
submission.head()

,user_id,prediction


In [38]:
submission.user_id = test.user_id
submission.head()

,user_id,prediction
0,57,NaN
1,58,NaN
2,59,NaN
3,60,NaN
4,61,NaN


In [39]:
submission.prediction = cat_p
submission

,user_id,prediction
0,57,0
1,58,0
2,59,0
3,60,0
4,61,0
...,...,...
24143,482942,0
24144,482943,0
24145,482944,0
24146,482945,0


In [40]:
submission.to_csv('submission.csv', index=False)